In [ ]:
#pip install -U googlemaps

In [3]:
import pandas as pd
import re
import googlemaps
import numpy as np
import ast
import datetime

# GOOGLE

In [4]:
Restaurantes=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Google.csv')

In [5]:
Restaurantes.head(1)

,name,address,gmap_id,description,latitude,longitude,category,price,hours,MISC,url
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",0x88008ff5b3c52fd3:0xdbe482cf9fb221db,NaN,44.306954,-89.845783,['Restaurant'],NaN,"[['Thursday', '12–11PM'], ['Friday', '12PM–12A...","{'Service options': ['Takeout', 'Dine-in', 'De...",https://www.google.com/maps/place//data=!4m2!3...


#### Generar Tabla Datos_Geograficos

In [13]:
Datos_Geograficos=Restaurantes[['gmap_id','latitude','longitude']].copy()

In [14]:
# Define tu clave de API de Google Maps.

with open('../PROYECTO_EQUIPO/api.txt', 'r') as archivo:
    for linea in archivo:
        a = linea  
gmaps = googlemaps.Client(key=a)

In [15]:
# Función para obtener la dirección, ciudad, estado y CP a partir de la latitud y longitud.
def obtener_direccion(latitud, longitud):
    # Hacer una solicitud de geocodificación inversa.
    resultado = gmaps.reverse_geocode((latitud, longitud))
    
    # Inicializar valores por defecto.
    direccion = ''
    ciudad = ''
    estado = ''
    cp = ''
    
    # Si hay resultados, buscar la información relevante.
    if resultado:
        primer_resultado = resultado[0]
        direccion = primer_resultado.get('formatted_address', '')
        for componente in primer_resultado['address_components']:
            tipos = componente.get('types', [])
            if 'locality' in tipos:
                ciudad = componente.get('long_name', '')
            elif 'administrative_area_level_1' in tipos:
                estado = componente.get('short_name', '')
            elif 'postal_code' in tipos:
                cp = componente.get('long_name', '')
    
    return direccion, ciudad, estado, cp

In [16]:
# Crear columnas vacías para almacenar los resultados.
Datos_Geograficos.insert(1, 'Direccion', '')
Datos_Geograficos.insert(2, 'Ciudad', '')
Datos_Geograficos.insert(3, 'Estado', '')
Datos_Geograficos.insert(4, 'CP', '')

# Iterar sobre las filas y actualizar las columnas con los resultados.
for index, row in Datos_Geograficos.iterrows():
    direccion, ciudad, estado, cp = obtener_direccion(row['latitude'], row['longitude'])
    Datos_Geograficos.at[index, 'Direccion'] = direccion
    Datos_Geograficos.at[index, 'Ciudad'] = ciudad
    Datos_Geograficos.at[index, 'Estado'] = estado
    Datos_Geograficos.at[index, 'CP'] = cp

# Ahora, el DataFrame "df" contiene las columnas de dirección, ciudad, estado y CP con la información correspondiente.

In [76]:
Datos_Geograficos.sample(1)

,gmap_id,Direccion,Ciudad,Estado,CP,latitude,longitude
916,0x8805a8a3e680fb01:0x8d543ed5ee91e049,"255 W Broadway, Waukesha, WI 53186, USA",Waukesha,WI,53186,43.011254,-88.231085


In [18]:
#Tabla Datos_Geográficos
Datos_Geograficos.to_csv('Datos_GeograficosG.csv')

### Category

In [41]:
Category_copy= Restaurantes.copy()

In [30]:
# Supongamos que tienes un DataFrame llamado DF1 con una columna 'category' que contiene los datos en formato de cadena.

# Obtén los datos únicos de la columna 'category'
categorias_unicas = set()
for categoria in Category_copy['category']:
    categorias = ast.literal_eval(categoria)  # Convierte la cadena en una lista
    categorias_unicas.update(categorias)

categorias_unicas = list(categorias_unicas)

# Crea un DataFrame vacío con las categorías únicas como columnas
df_Category = pd.DataFrame(columns=categorias_unicas)

# Itera a través de las filas de DF1 y llena el nuevo DataFrame
for index, row in Category_copy.iterrows():
    categorias = ast.literal_eval(row['category'])  # Convierte la cadena en una lista
    for categoria in categorias:
        df_Category.at[index, categoria] = 1

# Rellena los valores faltantes con 0
df_Category = df_Category.fillna(0)

# Agrega la columna 'gmap_id' al nuevo DataFrame
df_Category['gmap_id'] = Category_copy['gmap_id'].copy()

/var/folders/gg/08x9mdfn43q02625840rv79h0000gn/T/ipykernel_79851/4031128395.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_Category['gmap_id'] = Category_copy['gmap_id'].copy()


In [80]:
df_Category.sample(1)

,gmap_id,Linens store,Motel,Hunting club,Japanese language instructor,Lounge,Beer hall,Food products supplier,African restaurant,Bar,...,Mediterranean restaurant,Ice supplier,Sports bar,Children's party service,Repair service,Kosher grocery store,Cookie shop,Self-catering accommodation,Breakfast restaurant,Wedding venue
1156,0x89e2ee543296b24b:0x9e796dea707df43c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#Tabla Categorización de Restaurantes
df_Category.to_csv('CategorizacionG.csv')

In [14]:
# Obtiene las 10 categorías principales
suma_por_categoria = df_Category.sum()
top_10_categorias = suma_por_categoria.nlargest(20)
print("Las 10 categorías principales son:")
print(top_10_categorias)

Las 10 categorías principales son:
Restaurant              1550
Fast food restaurant     561
Takeout Restaurant       496
Pizza restaurant         430
Sandwich shop            385
Caterer                  329
American restaurant      284
Mexican restaurant       240
Delivery Restaurant      177
Breakfast restaurant     170
Bar                      169
Chinese restaurant       162
Pizza delivery           147
Coffee shop              138
Italian restaurant       126
Pizza Takeout            125
Cafe                     106
Bar & grill              105
Gas station              103
Family restaurant        100
dtype: int64


### Cualidades (MISC)

In [264]:

MISC_copia = Restaurantes[['gmap_id','MISC']].copy()

In [265]:
MISC_copia['MISC'] = MISC_copia['MISC'].fillna('{}').apply(ast.literal_eval) #Completamos los valores faltantes en esta columna con '{}'

In [266]:
#Combierte la columna en una lista de diccionarios y json_normalize pasa los valores a 
#columnas
df_expanded = pd.json_normalize(MISC_copia['MISC'].tolist()) 
#Combinamos el df original con el nuevo que tiene las columnas separadas
MISC_df = pd.concat([MISC_copia.drop('MISC', axis=1), df_expanded], axis=1) 

In [267]:
MISC_df=MISC_df.set_index('gmap_id')

In [268]:
# Función para extraer el texto de una lista de cadenas
def extraer_texto(lista):
    if isinstance(lista, list):
        return ', '.join([item for item in lista if isinstance(item, str)])
    return np.NaN

# Aplicar la función a todas las celdas del DataFrame usando applymap
MISC_df = MISC_df.applymap(extraer_texto)

In [269]:
MISC_df.head(1)

,Service options,Health & safety,Highlights,Popular for,Accessibility,Offerings,Amenities,Atmosphere,Crowd,Dining options,Payments,Planning,From the business,Health and safety
gmap_id,,,,,,,,,,,,,,
0x88008ff5b3c52fd3:0xdbe482cf9fb221db,"Takeout, Dine-in, Delivery",Staff required to disinfect surfaces between v...,"Great beer selection, Great cocktails","Lunch, Dinner, Solo dining",Wheelchair accessible entrance,"Alcohol, Beer, Cocktails, Coffee, Comfort food...","Bar onsite, Good for kids, High chairs, Restroom","Casual, Cozy","Groups, Tourists",NaN,NaN,NaN,NaN,NaN


In [252]:
MISC_df.to_csv('CualidadesG.csv')

### Horarios

In [270]:
Horarios=Restaurantes[['gmap_id','hours']].copy()

In [271]:
Horarios['hours'].isna().sum()

230

In [272]:
Horarios['hours'].fillna("[['Thursday', 'Horario no disponible'], ['Friday', 'Horario no disponible'], ['Saturday', 'Horario no disponible'], ['Sunday', 'Horario no disponible'], ['Monday', 'Horario no disponible'],['Tuesday', 'Horario no disponible'], ['Wednesday', 'Horario no disponible']]", inplace=True)

In [273]:
dataframes_individuales = []

for _, row in Horarios.iterrows():
    try:
        lista_horarios = ast.literal_eval(row['hours'])
        df = pd.DataFrame(lista_horarios, columns=['Dia', 'Horario'])
        df['gmap_id'] = row['gmap_id']
        dataframes_individuales.append(df)
    except (ValueError, SyntaxError):
        pass  # Ignora las filas con valores NaN o cadenas mal formadas.

# Concatena todos los DataFrames individuales en uno solo.
tabla_pivot_total = pd.concat(dataframes_individuales)

# Pivotea la tabla.
Tabla_Horarios = tabla_pivot_total.pivot(index='gmap_id', columns='Dia', values='Horario')

In [274]:
Tabla_Horarios.sample(2)

Dia,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
gmap_id,,,,,,,
0x89ba97e95fbe787f:0xd085ed3ba7a2ff3d,11AM–8PM,11AM–8PM,11AM–8PM,Closed,11AM–8PM,11AM–8PM,11AM–8PM
0x80c8c3d564da353b:0xd67c218f201a72dd,5PM–12AM,5–10PM,5PM–12AM,5–10PM,5–10PM,5–10PM,5–10PM


In [275]:
Tabla_Horarios.to_csv('HorariosG.csv')

### RESTAURANTES

In [ ]:
#Dejar la tabla principal de restaurantes

In [47]:
Atributos_Restaurantes=Restaurantes[['gmap_id','name','description']].copy()

In [51]:
Atributos_Restaurantes.shape

(3223, 3)

In [50]:
Atributos_Restaurantes['description'].isna().sum()

2162

In [48]:
Atributos_Restaurantes.head(2)

,gmap_id,name,description
0,0x88008ff5b3c52fd3:0xdbe482cf9fb221db,Roux's Roadhouse 73,NaN
1,0x80c8db636d525e3b:0x6a51cce4b2dcc350,Crepes n' Tacos Mexican Grill,NaN


### Reviews Google

In [74]:
ReviewsG=pd.read_csv('../PROYECTO_EQUIPO/Restaurantes_Google_Reviews.csv')

In [81]:
ReviewsG.head(1)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,115209325040226198552,Erika Beck,2005-06-09,4,NaN,NaN,NaN,0x80c8c18a51117251:0xfcb58dd8308dbdfb
